# Eficiencia Julia

## Julia es rápido

Muy a menudo, técnicas de comparación o punt de referencia  (_benchmark_), son utilizadas para comparar lenguajes de programación. Estos puntos de referencia pueden dar lugar a largas discusiones, primero sobre qué se está evaluando exactamente y, en segundo lugar, qué explica las diferencias. Estas simples preguntas a veces pueden volverse más complicadas de que podemos imaginar al principio.

El propósito de los siguientes ejempos es mostrar un ejercicio de _benchmark_ sencillo y comprensible.

Varios julianos distinguidos han mostrado este ejercicios de microbenchmarks para mostrar la eficiencia de Julia, agradecemos sus explicaciones, la versión original de este matrial nace como una maravillosa conferencia de Steven Johnson en el MIT: ["Boxes and registers"](https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb)

## Contenidos

- Definir la función `sum`
- Implementaciones y _comparativas_ de `sum`
    - Julia(built-in)
    - Julia(hand-writen)
    - C (hand-writen)
    - Python (built-in)
    - Python (numpy)
    - Python (hand-writen)
- Usando paralelismo con Julia
    - Utilización de la asociatividad de punto flotante
    - Utilización de 4 cores: built-in
    - Utilización de 4 cores: hand-writen
- Resumen de las comparativas


## `sum`: una función fácil de entender

Consideremos la función que suma elementos de un vector `sum(a)`, la cual se calcula:
$$sum(a) = \sum_{i=1}^n a_i$$ donde $n$ es el número de elementos (longitud) de $a$.

In [1]:
a = rand(10^7)    #vector 1D de números aleatorios uniformes [0,1)

10000000-element Array{Float64,1}:
 0.985028095699813
 0.9582282377093105
 0.43174868268024613
 0.7159704867326904
 0.6194457051732758
 0.028170734043372336
 0.38605048333425396
 0.6174268363845778
 0.46151631533624204
 0.4692483217585155
 0.02950211010532766
 0.7363443826877325
 0.15664177702373183
 ⋮
 0.2831776549490781
 0.791802337250813
 0.5540134091467164
 0.9541021784747827
 0.25953608252036897
 0.2099217214498943
 0.20427430028048166
 0.5097462759863753
 0.22463319001701443
 0.07238776061382524
 0.04655644847927487
 0.9033117393065531

In [2]:
sum(a)

4.999840508477772e6

## Comparativas entre algunos cuantos lenguajes

In [3]:
@time sum(a)

  0.008131 seconds (1 allocation: 16 bytes)


4.999840508477772e6

In [4]:
@time sum(a)

  0.007728 seconds (1 allocation: 16 bytes)


4.999840508477772e6

In [5]:
@time sum(a)

  0.012141 seconds (1 allocation: 16 bytes)


4.999840508477772e6

La macro `@` puede devolver ciertos resultados sesgados, por tanto no es la mejor elección que hagamos. En su lugar utilizaremos el paquete `BenchmarkTools.jl` para hacer comparativas fáciles y mas precisas.

In [6]:
#using Pkg
#Pkg.add("BenchmarkTools")

using BenchmarkTools

In [7]:
@benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     6.102 ms (0.00% GC)
  median time:      8.101 ms (0.00% GC)
  mean time:        10.179 ms (0.00% GC)
  maximum time:     40.605 ms (0.00% GC)
  --------------
  samples:          491
  evals/sample:     1

## 1. Julia (built-in)

Bien, ya hemos visto el uso de la función predefinida `sum()`Julia. Por supuesto, está escrita en Julia, pero ¿funcionaría si escribiéramos una implementación nosotros mismos?

In [8]:
@which sum(a)

sum(a::AbstractArray; dims) in Base at reducedim.jl:722

Guardemos estos resultados de referencia en un diccionario para que podamos comenzar a realizar un seguimiento de ellos y compararlos en el futuro.

In [9]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     6.048 ms (0.00% GC)
  median time:      8.001 ms (0.00% GC)
  mean time:        9.157 ms (0.00% GC)
  maximum time:     61.043 ms (0.00% GC)
  --------------
  samples:          552
  evals/sample:     1

In [10]:
j_bench.times

552-element Array{Float64,1}:
 6.048153e6
 6.07115e6
 6.079339e6
 6.109084e6
 6.179283e6
 6.200633e6
 6.223306e6
 6.232873e6
 6.247611e6
 6.26318e6
 6.306321e6
 6.331589e6
 6.337517e6
 ⋮
 1.8475785e7
 1.8726576e7
 1.89963e7
 1.9538026e7
 2.0873495e7
 2.1026122e7
 2.2630816e7
 2.3032203e7
 2.746178e7
 3.0291554e7
 3.2678857e7
 6.1042845e7

In [11]:
d = Dict()
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 1 entry:
  "Julia built-in" => 6.04815

## 2. Julia (hand-written)

In [12]:
function mysum(A)
    s = 0.0
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [13]:
mysum(a)

4.999840508478023e6

In [14]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     15.330 ms (0.00% GC)
  median time:      16.908 ms (0.00% GC)
  mean time:        18.322 ms (0.00% GC)
  maximum time:     54.487 ms (0.00% GC)
  --------------
  samples:          273
  evals/sample:     1

In [15]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "Julia hand-written" => 15.3302
  "Julia built-in"     => 6.04815

## 3. Lenguaje C

C a menudo se considera el estándar de oro: difícil para el ser humano, agradable para la máquina. Lograr estar dentro de un factor de ~2xC a menudo es satisfactorio. No obstante, incluso dentro de C, hay muchos tipos de optimizaciones posibles de las que un desarrollador de C novato puede o no aprovechar.

El autor actual no habla C, por lo que no comprende la celda de abajo, pero está feliz de saber que puede poner código C en una sesión de Julia, compilarlo y ejecutarlo. Tengamos en cuenta que `"""` envuelve una cadena de varias líneas.

In [16]:
using Libdl
C_code = """
    #include <stddef.h>
    double c_sum(size_t n, double *X) {
        double s = 0.0;
        for (size_t i = 0; i < n; ++i) {
            s += X[i];
        }
        return s;
    }
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [17]:
c_sum(a)

4.999840508478023e6

In [18]:
c_bench = @benchmark c_sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     15.531 ms (0.00% GC)
  median time:      17.586 ms (0.00% GC)
  mean time:        20.616 ms (0.00% GC)
  maximum time:     108.382 ms (0.00% GC)
  --------------
  samples:          243
  evals/sample:     1

In [19]:
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 3 entries:
  "C"                  => 15.531
  "Julia hand-written" => 15.3302
  "Julia built-in"     => 6.04815

## 4. Python (built in `sum`)

El paquete `PyCall` provee una interfase en Julia para usar Python

In [20]:
using PyCall

In [21]:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [22]:
pysum(a)

4.999840508478023e6

In [23]:
py_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     925.158 ms (0.00% GC)
  median time:      1.020 s (0.00% GC)
  mean time:        1.012 s (0.00% GC)
  maximum time:     1.110 s (0.00% GC)
  --------------
  samples:          5
  evals/sample:     1

In [24]:
d["Python built.in"] = minimum(py_bench.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "C"                  => 15.531
  "Julia hand-written" => 15.3302
  "Python built.in"    => 925.158
  "Julia built-in"     => 6.04815

## 5. Python Numpy

`Numpy` es una biblioteca en C optimizada y se llama directamente desde Python:

In [26]:
using Pkg
Pkg.add("Conda")
using Conda

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 % 1.0 %Fetching: [=====>                                   ]  10.9 %Fetching: [======>                                  ]  12.9 %Fetching: [========>                                ]  18.8 %                           ]  31.0 %Fetching: [==============>                          ]  34.7 %>                     ]  46.9 %Fetching: [=====================>                   ]  51.0 %.4 %        ]  79.6 %Fetching: [======================================>  ]  94.0 %Fetching: [=======================================> ]  96.2 %

  Resolving package versions...
  Installed Conda ─ v1.5.2
Updating `~/.julia/environments/v1.5/Project.toml`
  [8f4d0f93] + Conda v1.5.2
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [8f4d0f93] ↑ Conda v1.5.0 ⇒ v1.5.2
  [8f1865be] ↑ ZeroMQ_jll v4.3.2+5 ⇒ v4.3.2+6
  [a9144af2] + libsodium_jll v1.0.19+0
   Building Conda → `~/.julia/packages/Conda/sNGum/deps/build.log`


In [27]:
Conda.add("numpy")

┌ Info: Running `conda install -y numpy` in root environment
└ @ Conda /home/oscar/.julia/packages/Conda/x5ml4/src/Conda.jl:115


Solving environment: ...working... done

## Package Plan ##

  environment location: /home/oscar/.julia/conda/3

  added / updated specs:
    - numpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.7.5   |       h06a4308_1         113 KB
    certifi-2021.5.30          |   py38h06a4308_0         138 KB
    conda-4.10.3               |   py38h06a4308_0         2.9 MB
    openssl-1.1.1k             |       h27cfd23_0         2.5 MB
    ------------------------------------------------------------
                                           Total:         5.7 MB

The following packages will be UPDATED:

  ca-certificates                              2020.10.14-0 --> 2021.7.5-h06a4308_1
  certifi            pkgs/main/noarch::certifi-2020.6.20-p~ --> pkgs/main/linux-64::certifi-2021.5.30-py38h06a4308_0
  conda                                4.9.2-py38h06a4308_0 --> 4.10.3-

In [28]:
numpy_sum = pyimport("numpy")["sum"]

PyObject <function sum at 0x7f83ad2ca6a8>

In [29]:
py_numpy_bench = @benchmark $numpy_sum($a)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     5.922 ms (0.00% GC)
  median time:      9.456 ms (0.00% GC)
  mean time:        10.664 ms (0.00% GC)
  maximum time:     37.433 ms (0.00% GC)
  --------------
  samples:          469
  evals/sample:     1

In [30]:
numpy_sum(a)

4.9998405084777735e6

In [31]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "C"                  => 15.531
  "Julia hand-written" => 15.3302
  "Python built.in"    => 925.158
  "Python numpy"       => 5.92153
  "Julia built-in"     => 6.04815

# 6. Python (hand-written)

In [32]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x7f83b4051400>

In [33]:
py_hand = @benchmark $sum_py($a)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     1.079 s (0.00% GC)
  median time:      1.171 s (0.00% GC)
  mean time:        1.509 s (0.00% GC)
  maximum time:     2.773 s (0.00% GC)
  --------------
  samples:          5
  evals/sample:     1

In [34]:
sum_py(a)

4.999840508478023e6

In [35]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "C"                   => 15.531
  "Julia hand-written"  => 15.3302
  "Python built.in"     => 925.158
  "Python numpy"        => 5.92153
  "Python hand-written" => 1079.14
  "Julia built-in"      => 6.04815

## Resumen

In [36]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Python numpy................5.9
Julia built-in..............6.0
Julia hand-written.........15.3
C..........................15.5
Python built.in...........925.2
Python hand-written......1079.1


**Ejercicio:**
Implementar la multiplicación matriz-vector $M \times V$ donde $M$ es una matriz y $V$ un vector, ambos con las dimensiones adecuadas. Envolver el cálculo en una función `multmatvec` y calcular los tiempos de ejecución con `@benchmark`.